In [340]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import joblib

print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("tensorflow version:", tf.__version__)
print("matplotlib version:", matplotlib.__version__)  # Access __version__ from matplotlib

pandas version: 2.2.2
numpy version: 2.0.2
tensorflow version: 2.19.0
matplotlib version: 3.10.0


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

## course.csv


In [157]:
data_path_course = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/courses.csv"
df_course = pd.read_csv(data_path_course, header=None)
df_course.head()

,0,1,2
0,code_module,code_presentation,module_presentation_length
1,AAA,2013J,268
2,AAA,2014J,269
3,BBB,2013J,268
4,BBB,2014J,262


## assessments.csv

In [158]:
data_path_assessments = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/assessments.csv"
df_assessments = pd.read_csv(data_path_assessments)
df_assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0


## studentAssessment.csv

In [159]:
data_path_studentAssessment = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/studentAssessment.csv"
df_studentAssessment = pd.read_csv(data_path_studentAssessment)
df_studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78
1,1752,28400,22,0,70
2,1752,31604,17,0,72
3,1752,32885,26,0,69
4,1752,38053,19,0,79


## studentInfo.csv

In [160]:
data_path_studentInfo = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/studentInfo.csv"
df_studentInfo = pd.read_csv(data_path_studentInfo)
df_studentInfo.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


## studentRegistration.csv

In [161]:
data_path_studentRegistration = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/studentRegistration.csv"
df_studentRegistration = pd.read_csv(data_path_studentRegistration)
df_studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159,?
1,AAA,2013J,28400,-53,?
2,AAA,2013J,30268,-92,12
3,AAA,2013J,31604,-52,?
4,AAA,2013J,32885,-176,?


## studentVle.csv

In [162]:
data_path_studentVle = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/studentVle.csv"
df_studentVle = pd.read_csv(data_path_studentVle)
df_studentVle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


## vle.csv

In [163]:
data_path_vle = "/content/drive/MyDrive/Skripsi/open+university+learning+analytics+dataset/vle.csv"
df_vle = pd.read_csv(data_path_vle)
df_vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,?,?
1,546712,AAA,2013J,oucontent,?,?
2,546998,AAA,2013J,resource,?,?
3,546888,AAA,2013J,url,?,?
4,547035,AAA,2013J,resource,?,?


rekap-volunter-28-agustus.csv

In [164]:
data_path_vle = "/content/drive/MyDrive/Skripsi/dataset/rekap-volunter-28-agustus.csv.xlsx - rekap-volunter-28-agustus.csv"
df_rekap_volunteer = pd.read_csv(data_path_vle)
df_rekap_volunteer.head()

,NAMA,NAMA_DEPT,npm,time_materi_video,time_materi_dokumen,time_materi_artikel,time_task,time_quiz,time_material,time_forum,total_time,time
0,Abi Rizki Hidayat,Sistem Informasi,882006580948,1199,7461,5262,9016,0,13922,5,36865,10:14:25
1,Abimantrana Agra Prartana,Bisnis Digital,81246862329,10051,21252,11170,65,0,42473,0,85011,23:36:51
2,Achmad Sahlan,Teknologi Informasi,882006661211,7520,1155,607,98,0,9282,0,18662,5:11:02
3,Adinda Rizkya Putri Wibawa,Ilmu Komunikasi,82220754285,190,5352,1940,5,0,7482,0,14969,4:09:29
4,ADITYA SAPUTRA DARMAWAN,Sistem Informasi,82241354470,68,145,183,10,0,396,0,802,0:13:22


dfjadi.csv

In [165]:
data_path_vle = "/content/drive/MyDrive/Skripsi/dataset/dfjadi-simplified - dfjadi-simplified.csv"
df_jadi = pd.read_csv(data_path_vle)
df_jadi.head()

,NIM,Nama,Jenis Kelamin,Prodi,Skor 1,Skor 2,Skor 3,Skor 4,Pemrosesan,Persepsi,Input,Pemahaman
0,22SA11A074,Mohamad Ridho Mubarok,Laki-laki,Informatika,-1,1,-3,-3,Reflektif Rendah,Sensing Rendah,Verbal Rendah,Global Rendah
1,22SA11A134,Maria Angelina Cahyani Candrakasih,Perempuan,Informatika,1,-3,-5,1,Aktif Rendah,Intuitif Rendah,Verbal Sedang,Sequential Rendah
2,22SA31A027,Tresnanda Agsifa Cakra Buana,Laki-laki,Teknologi Informasi,-5,5,1,1,Reflektif Sedang,Sensing Sedang,Visual Rendah,Sequential Rendah
3,22sa11a038,Ndaru Febrian Pujo Leksono,Laki-laki,Informatika,-1,1,-7,-3,Reflektif Rendah,Sensing Rendah,Verbal Sedang,Global Rendah
4,22SA11A035,Nurul Khotimah Ansyari,Perempuan,Informatika,-5,-1,-5,3,Reflektif Sedang,Intuitif Rendah,Verbal Sedang,Sequential Rendah


rekap.csv

In [166]:
data_path_vle = "/content/drive/MyDrive/Skripsi/dataset/rekap-001.csv"
df_rekap = pd.read_csv(data_path_vle)
df_rekap.head()

,NPM,NAMA,time_materials,time_tasks,time_forums,time_quizzes
0,MHS001,Mahasiswa Pertama,0,0,0,0
1,21SA1090,Bobit,0,0,5,0
2,MHSIF001,Mahasiswa IF 1,19636,1009,2959,0
3,MHSIF002,Mahasiswa IF 2,0,0,0,0
4,MHSIF003,Mahasiswa IF 3,0,0,0,0


In [167]:
df_jadi_selected = df_jadi[["NIM", "Pemrosesan", "Persepsi", "Input", "Pemahaman"]]

df_merged = pd.merge(
    df_rekap,
    df_jadi_selected,
    left_on="NPM",
    right_on="NIM",
    how="left"   # atau "inner" tergantung kebutuhan
)
df_merged.head(10)

,NPM,NAMA,time_materials,time_tasks,time_forums,time_quizzes,NIM,Pemrosesan,Persepsi,Input,Pemahaman
0,MHS001,Mahasiswa Pertama,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,21SA1090,Bobit,0,0,5,0,NaN,NaN,NaN,NaN,NaN
2,MHSIF001,Mahasiswa IF 1,19636,1009,2959,0,NaN,NaN,NaN,NaN,NaN
3,MHSIF002,Mahasiswa IF 2,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,MHSIF003,Mahasiswa IF 3,0,0,0,0,NaN,NaN,NaN,NaN,NaN
5,MHSIF004,Mahasiswa IF 4,0,0,0,0,NaN,NaN,NaN,NaN,NaN
6,MHSIF005,Mahasiswa IF 5,0,0,0,0,NaN,NaN,NaN,NaN,NaN
7,MHSIF006,Mahasiswa IF 6,0,0,0,0,NaN,NaN,NaN,NaN,NaN
8,MHSIF007,Mahasiswa IF 7,0,10,0,0,NaN,NaN,NaN,NaN,NaN
9,MHSIF008,Mahasiswa IF 8,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [168]:
df_jadi.to_csv("output.csv", index=False)

In [169]:
df_jadi.sort_values('NIM')

,NIM,Nama,Jenis Kelamin,Prodi,Skor 1,Skor 2,Skor 3,Skor 4,Pemrosesan,Persepsi,Input,Pemahaman
464,-,Malvino Abel Calvero,Laki-laki,Informatika,-1,-3,-5,-3,Reflektif Rendah,Intuitif Rendah,Verbal Sedang,Global Rendah
449,-,Azhar,Laki-laki,Ilmu Komunikasi,-3,5,-3,-1,Reflektif Rendah,Sensing Sedang,Verbal Rendah,Global Rendah
460,-,akbar dr,Laki-laki,Informatika,1,-3,-3,1,Aktif Rendah,Intuitif Rendah,Verbal Rendah,Sequential Rendah
459,-,Reghina salwa,Perempuan,Informatika,-5,-1,-5,-1,Reflektif Sedang,Intuitif Rendah,Verbal Sedang,Global Rendah
447,-,Tutur hidayah,Laki-laki,Informatika,-1,-1,3,-3,Reflektif Rendah,Intuitif Rendah,Visual Rendah,Global Rendah
...,...,...,...,...,...,...,...,...,...,...,...,...
455,_,panji suwarsono,Laki-laki,Informatika,-1,1,1,3,Reflektif Rendah,Sensing Rendah,Visual Rendah,Sequential Rendah
451,d,Dina reksa Saputri,Perempuan,Ilmu Komunikasi,-3,-5,-1,-1,Reflektif Rendah,Intuitif Sedang,Verbal Rendah,Global Rendah
130,tes,tes,Laki-laki,Informatika,-1,-3,-3,-1,Reflektif Rendah,Intuitif Rendah,Verbal Rendah,Global Rendah
437,y,Rizki nur rahman,Laki-laki,Ilmu Komunikasi,-1,3,-1,-1,Reflektif Rendah,Sensing Rendah,Verbal Rendah,Global Rendah


In [170]:
df_rekap.loc[98]

,98
NPM,MHSBD007
NAMA,Mahasiswa BD 7
time_materials,0
time_tasks,0
time_forums,0
time_quizzes,0


# Explorasi dfjadi

In [171]:
data_path_vle = "/content/drive/MyDrive/Skripsi/dataset/dfjadi-simplified - dfjadi-simplified.csv"
df_jadi = pd.read_csv(data_path_vle)
df_jadi.head()

,NIM,Nama,Jenis Kelamin,Prodi,Skor 1,Skor 2,Skor 3,Skor 4,Pemrosesan,Persepsi,Input,Pemahaman
0,22SA11A074,Mohamad Ridho Mubarok,Laki-laki,Informatika,-1,1,-3,-3,Reflektif Rendah,Sensing Rendah,Verbal Rendah,Global Rendah
1,22SA11A134,Maria Angelina Cahyani Candrakasih,Perempuan,Informatika,1,-3,-5,1,Aktif Rendah,Intuitif Rendah,Verbal Sedang,Sequential Rendah
2,22SA31A027,Tresnanda Agsifa Cakra Buana,Laki-laki,Teknologi Informasi,-5,5,1,1,Reflektif Sedang,Sensing Sedang,Visual Rendah,Sequential Rendah
3,22sa11a038,Ndaru Febrian Pujo Leksono,Laki-laki,Informatika,-1,1,-7,-3,Reflektif Rendah,Sensing Rendah,Verbal Sedang,Global Rendah
4,22SA11A035,Nurul Khotimah Ansyari,Perempuan,Informatika,-5,-1,-5,3,Reflektif Sedang,Intuitif Rendah,Verbal Sedang,Sequential Rendah


In [172]:
df_start_with_21 = df_jadi[df_jadi['NIM'].astype(str).str.startswith('21')]
# df_start_with_21.head()
df_start_with_21.NIM.count()

np.int64(15)

In [173]:
df_start_with_22 = df_jadi[df_jadi['NIM'].astype(str).str.startswith('22')]
# df_start_with_22.head()
df_start_with_22.NIM.count()

np.int64(150)

In [174]:
df_start_with_23 = df_jadi[df_jadi['NIM'].astype(str).str.startswith('23')]
# df_start_with_23.head()
df_start_with_23.NIM.count()

np.int64(252)

In [175]:
df_start_with_24 = df_jadi[df_jadi['NIM'].astype(str).str.startswith('24')]
# df_start_with_24.head()
df_start_with_24.NIM.count()

np.int64(146)

In [176]:
df_jadi.NIM.count()

np.int64(604)

In [177]:
df_jadi.NIM.count() - (df_start_with_21.NIM.count() + df_start_with_22.NIM.count() + df_start_with_23.NIM.count() + df_start_with_24.NIM.count())

np.int64(41)

In [178]:
df_merged_years = pd.concat([df_start_with_21, df_start_with_22, df_start_with_23, df_start_with_24])
# display(df_merged_years.head())
df_merged_years.NIM.count()

np.int64(563)

In [179]:
df_merged_years.head()

,NIM,Nama,Jenis Kelamin,Prodi,Skor 1,Skor 2,Skor 3,Skor 4,Pemrosesan,Persepsi,Input,Pemahaman
136,21SA1194,Rizky Aulia Rahman,Laki-laki,Informatika,-1,5,-5,-7,Reflektif Rendah,Sensing Sedang,Verbal Sedang,Global Sedang
168,21SA1086,Muhammad Thoriq Jamil,Laki-laki,Informatika,-3,7,-3,1,Reflektif Rendah,Sensing Sedang,Verbal Rendah,Sequential Rendah
171,21SA1158,Velizha Sandy Kusuma,Perempuan,Informatika,-3,-5,1,-3,Reflektif Rendah,Intuitif Sedang,Visual Rendah,Global Rendah
173,21SA1220,Aldona Septiana,Perempuan,Informatika,-5,-1,-1,-3,Reflektif Sedang,Intuitif Rendah,Verbal Rendah,Global Rendah
217,21SB1022,Refita Dwi Nurjanah,Perempuan,Bisnis Digital,1,-1,-5,-1,Aktif Rendah,Intuitif Rendah,Verbal Sedang,Global Rendah


In [180]:
df_merged_years['Pemrosesan'].isna().sum()

np.int64(0)

In [181]:
df_merged_years.to_csv("df_merged_years_output.csv", index=False)

# Explorasi Mahasiswa Spend Time

In [182]:
data_path_vle = "/content/drive/MyDrive/Skripsi/dataset/rekap-001.csv"
df_rekap = pd.read_csv(data_path_vle)
df_rekap.head()

,NPM,NAMA,time_materials,time_tasks,time_forums,time_quizzes
0,MHS001,Mahasiswa Pertama,0,0,0,0
1,21SA1090,Bobit,0,0,5,0
2,MHSIF001,Mahasiswa IF 1,19636,1009,2959,0
3,MHSIF002,Mahasiswa IF 2,0,0,0,0
4,MHSIF003,Mahasiswa IF 3,0,0,0,0


In [183]:
df_rekap.NPM.count()

np.int64(456)

# Dataset

In [316]:
df_merged_years = df_jadi[["NIM", "Nama", "Pemrosesan", "Persepsi", "Input", "Pemahaman"]]

df_merged = pd.merge(
    df_merged_years,
    df_rekap,
    left_on="NIM",
    right_on="NPM",
    how="left"   # atau "inner" tergantung kebutuhan
)
df_merged.head()

,NIM,Nama,Pemrosesan,Persepsi,Input,Pemahaman,NPM,NAMA,time_materials,time_tasks,time_forums,time_quizzes
0,22SA11A074,Mohamad Ridho Mubarok,Reflektif Rendah,Sensing Rendah,Verbal Rendah,Global Rendah,NaN,NaN,NaN,NaN,NaN,NaN
1,22SA11A134,Maria Angelina Cahyani Candrakasih,Aktif Rendah,Intuitif Rendah,Verbal Sedang,Sequential Rendah,NaN,NaN,NaN,NaN,NaN,NaN
2,22SA31A027,Tresnanda Agsifa Cakra Buana,Reflektif Sedang,Sensing Sedang,Visual Rendah,Sequential Rendah,NaN,NaN,NaN,NaN,NaN,NaN
3,22sa11a038,Ndaru Febrian Pujo Leksono,Reflektif Rendah,Sensing Rendah,Verbal Sedang,Global Rendah,NaN,NaN,NaN,NaN,NaN,NaN
4,22SA11A035,Nurul Khotimah Ansyari,Reflektif Sedang,Intuitif Rendah,Verbal Sedang,Sequential Rendah,NaN,NaN,NaN,NaN,NaN,NaN


In [317]:
df_merged.NPM.count()

np.int64(123)

In [318]:
df_merged['Pemrosesan'].isna().sum()

np.int64(0)

In [319]:
df_merged['Pemrosesan'].notna().sum()

np.int64(604)

In [320]:
df_merged.Pemrosesan.nunique()

5

In [321]:
df_merged.Pemrosesan.unique()

array(['Reflektif Rendah', 'Aktif Rendah', 'Reflektif Sedang',
       'Reflektif Kuat', 'Aktif Sedang'], dtype=object)

In [322]:
df_merged.Input.unique()

array(['Verbal Rendah', 'Verbal Sedang', 'Visual Rendah', 'Visual Sedang',
       'Verbal Kuat'], dtype=object)

In [323]:
df_merged.Input.nunique()

5

In [324]:
# Pemrosesan: Refrektif, Aktif
# Persepsi: Visual, Verbal

In [325]:
def map_pemrosesan(row):
    if (row['Pemrosesan'] == 'Reflektif Kuat') or (row['Pemrosesan'] == 'Reflektif Sedang') or (row['Pemrosesan'] == 'Reflektif Rendah'):
        return 'Reflektif'
    elif (row['Pemrosesan'] == 'Aktif Kuat') or (row['Pemrosesan'] == 'Aktif Sedang') or (row['Pemrosesan'] == 'Aktif Rendah'):
        return 'Aktif'
    else:
        return None

def map_input(row):
    if (row['Input'] == 'Visual Kuat') or (row['Input'] == 'Visual Sedang') or (row['Input'] == 'Visual Rendah'):
        return 'Visual'
    elif (row['Input'] == 'Verbal Kuat') or (row['Input'] == 'Verbal Sedang') or (row['Input'] == 'Verbal Rendah'):
        return 'Verbal'
    else:
        return None

# Tambahkan kolom baru
df_merged['Pemrosesan'] = df_merged.apply(map_pemrosesan, axis=1)
df_merged['Input'] = df_merged.apply(map_input, axis=1)


In [326]:
df_merged.head()

,NIM,Nama,Pemrosesan,Persepsi,Input,Pemahaman,NPM,NAMA,time_materials,time_tasks,time_forums,time_quizzes
0,22SA11A074,Mohamad Ridho Mubarok,Reflektif,Sensing Rendah,Verbal,Global Rendah,NaN,NaN,NaN,NaN,NaN,NaN
1,22SA11A134,Maria Angelina Cahyani Candrakasih,Aktif,Intuitif Rendah,Verbal,Sequential Rendah,NaN,NaN,NaN,NaN,NaN,NaN
2,22SA31A027,Tresnanda Agsifa Cakra Buana,Reflektif,Sensing Sedang,Visual,Sequential Rendah,NaN,NaN,NaN,NaN,NaN,NaN
3,22sa11a038,Ndaru Febrian Pujo Leksono,Reflektif,Sensing Rendah,Verbal,Global Rendah,NaN,NaN,NaN,NaN,NaN,NaN
4,22SA11A035,Nurul Khotimah Ansyari,Reflektif,Intuitif Rendah,Verbal,Sequential Rendah,NaN,NaN,NaN,NaN,NaN,NaN


In [327]:
# df_merged.to_csv("df_merged_output.csv", index=False)

In [328]:
df_merged.columns.tolist()

['NIM',
 'Nama',
 'Pemrosesan',
 'Persepsi',
 'Input',
 'Pemahaman',
 'NPM',
 'NAMA',
 'time_materials',
 'time_tasks',
 'time_forums',
 'time_quizzes']

In [329]:
df_merged_simplified = df_merged[[
    'NIM',
    'NAMA',
    'time_materials',
    'time_tasks',
    'time_forums',
    'time_quizzes',
    'Pemrosesan',
    'Input',
    ]].copy()

In [330]:
df_merged_simplified.head()

,NIM,NAMA,time_materials,time_tasks,time_forums,time_quizzes,Pemrosesan,Input
0,22SA11A074,NaN,NaN,NaN,NaN,NaN,Reflektif,Verbal
1,22SA11A134,NaN,NaN,NaN,NaN,NaN,Aktif,Verbal
2,22SA31A027,NaN,NaN,NaN,NaN,NaN,Reflektif,Visual
3,22sa11a038,NaN,NaN,NaN,NaN,NaN,Reflektif,Verbal
4,22SA11A035,NaN,NaN,NaN,NaN,NaN,Reflektif,Verbal


In [331]:
df_merged_simplified['NAMA'].notna().sum()

np.int64(123)

In [332]:
df_merged_simplified_not_null =  df_merged_simplified[df_merged_simplified['NAMA'].notna()]
df_merged_simplified_not_null['NIM'].count()

np.int64(123)

In [333]:
df_merged_simplified_not_null.head()

,NIM,NAMA,time_materials,time_tasks,time_forums,time_quizzes,Pemrosesan,Input
16,23SA21A011,IWAN HARYATNO,49555.0,7610.0,5.0,0.0,Aktif,Visual
24,23SA11A045,REFIDA SEPTIANA PUTRI,36562.0,0.0,0.0,0.0,Reflektif,Verbal
29,22SA11A027,WISNU PRATAMA AJI,0.0,0.0,0.0,0.0,Reflektif,Visual
31,22SA11A289,MAHMUDI,14060.0,0.0,0.0,0.0,Reflektif,Verbal
36,22SA11A097,ADITYA OKTA SAPUTRA,2071.0,0.0,0.0,0.0,Reflektif,Verbal


## One Hot Encoding

In [334]:
df_merged_simplified_not_null['labels'] = df_merged_simplified_not_null[['Pemrosesan', 'Input']].values.tolist()

/tmp/ipython-input-1202623539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_simplified_not_null['labels'] = df_merged_simplified_not_null[['Pemrosesan', 'Input']].values.tolist()


In [335]:
df_merged_simplified_not_null.head()

,NIM,NAMA,time_materials,time_tasks,time_forums,time_quizzes,Pemrosesan,Input,labels
16,23SA21A011,IWAN HARYATNO,49555.0,7610.0,5.0,0.0,Aktif,Visual,"[Aktif, Visual]"
24,23SA11A045,REFIDA SEPTIANA PUTRI,36562.0,0.0,0.0,0.0,Reflektif,Verbal,"[Reflektif, Verbal]"
29,22SA11A027,WISNU PRATAMA AJI,0.0,0.0,0.0,0.0,Reflektif,Visual,"[Reflektif, Visual]"
31,22SA11A289,MAHMUDI,14060.0,0.0,0.0,0.0,Reflektif,Verbal,"[Reflektif, Verbal]"
36,22SA11A097,ADITYA OKTA SAPUTRA,2071.0,0.0,0.0,0.0,Reflektif,Verbal,"[Reflektif, Verbal]"


In [336]:
df_merged_simplified_not_null = df_merged_simplified_not_null.drop(columns=['NIM', 'NAMA', 'Pemrosesan', 'Input'])
df_merged_simplified_not_null.head()

,time_materials,time_tasks,time_forums,time_quizzes,labels
16,49555.0,7610.0,5.0,0.0,"[Aktif, Visual]"
24,36562.0,0.0,0.0,0.0,"[Reflektif, Verbal]"
29,0.0,0.0,0.0,0.0,"[Reflektif, Visual]"
31,14060.0,0.0,0.0,0.0,"[Reflektif, Verbal]"
36,2071.0,0.0,0.0,0.0,"[Reflektif, Verbal]"


In [348]:
df_merged_simplified_not_null.to_csv("df_merged_simplified_not_null_output.csv", index=False)

In [337]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df_merged_simplified_not_null["labels"])
X = df_merged_simplified_not_null.drop(columns=["labels"])

print("Label classes:", mlb.classes_)
print("Encoded labels:\n", y)

Label classes: ['Aktif' 'Reflektif' 'Verbal' 'Visual']
Encoded labels:
 [[1 0 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [1 0 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [1 0 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 0 1]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 0 1]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 

# Training

In [338]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [339]:
# 4️⃣ Model Random Forest + MultiOutput wrapper
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
multi_label_model = MultiOutputClassifier(base_model)
multi_label_model.fit(X_train, y_train)

# 5️⃣ Prediksi dan evaluasi
y_pred = multi_label_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

              precision    recall  f1-score   support

       Aktif       0.50      0.10      0.17        10
   Reflektif       0.74      0.96      0.84        27
      Verbal       0.85      0.90      0.88        31
      Visual       0.25      0.17      0.20         6

   micro avg       0.76      0.76      0.76        74
   macro avg       0.59      0.53      0.52        74
weighted avg       0.71      0.76      0.71        74
 samples avg       0.76      0.76      0.76        74



# Save

In [341]:
# Simpan model
joblib.dump(multi_label_model, "multi_label_model.pkl")

# Simpan juga encoder label
joblib.dump(mlb, "label_encoder.pkl")

['label_encoder.pkl']

# Load

In [352]:
# Load kembali model dan encoder
multi_label_model = joblib.load("multi_label_model.pkl")
mlb = joblib.load("label_encoder.pkl")

# Contoh input baru
X_new = [[7510.0,	0.0,	5.0,	0.0]]

# Prediksi
y_pred = multi_label_model.predict(X_new)

# Konversi hasil kembali ke label asli
decoded_labels = mlb.inverse_transform(y_pred)
print(decoded_labels)


[('Reflektif', 'Verbal')]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
